# MWS Secondary Target Selection

## MWS_SPECIAL_WDBINARY_SV

Maintainers:
  - Boris Gaensicke (selection/catalog)
  - Andrew Cooper (secondary target code)
  
See `/project/projectdirs/desi/target/secondary/README` for output data model. 

In [2]:
!pwd
!ls ../raw

/global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_WDBINARY_SV/1/code
WD_BinaryCandidates_GALEX_Gaia.csv  WD_BinaryCandidates_Variability.csv


In [3]:
# Parameters

target_class    = 'MWS_SPECIAL_WDBINARY_SV'
input_filenames = 'WD_BinaryCandidates_GALEX_Gaia.csv', 'WD_BinaryCandidates_Variability.csv'

OVERRIDE       = True
REF_CAT        = 'G'
REF_EPOCH      = 2015.5

colmap = {
    'ra':        'RA',
    'dec':       'DEC',
    'pmra':      'PMRA',
    'pmdec':     'PMDEC',
    'source_id': 'REF_ID'
}

Two target lists of WD binary candidates, one based on a GALEX + Gaia selection, and one which is a heterogeneous collection of variability-selected sources compiled from the literature, variable star observers, and the various time-domain surveys (ASAS-SN, CRTS, Gaia alerts).

Of those CVs/CV candidates, ~1280 are in the GALEX/Gaia cross-match. We use those as fiducial sample.

A selection on parallax significance, FUV magnitude, and G-band magnitude does a good job at tracing the outline of this fiducial sample:

parallax_over_error > 2 && FUV < 21 && phot_g_mean_mag > 16

=> 128956 sources, ~5 per square degree (see the sky map on https://desi.lbl.gov/trac/wiki/MilkyWayWG/TargetSelectionDoc/SpecialTarget)

Note that this selection includes a good fraction (~1/3) of single white dwarfs (lower left in the HRD), which will be targeted separately as flux calibration and science targets. These could be relatively easily be removed via their Gaia IDs, if necessary.

In [4]:
from   importlib import reload
import sys
PYPATH = '/project/projectdirs/desi/external/mws/py/'
if PYPATH not in sys.path: sys.path.append(PYPATH)
import mwsecondary.mwsecondary as mws
reload(mws)

import os
import sys
import numpy as np

targets = mws.SecondaryTargets(target_class,input_filenames,ref_cat=REF_CAT,ref_epoch=REF_EPOCH)
targets.print_info()
mask    = targets.select_finite_range('dec',lower_limit=-30.0,verbose=True)
mask   &= targets.select_unique('source_id',verbose=True)

# Adjust to secondary target data model and write output
output = targets.get_output(mask,cols=colmap,write=True,override=OVERRIDE)
print(output[0:3])

Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_WDBINARY_SV/1/raw/WD_BinaryCandidates_GALEX_Gaia.csv
Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_WDBINARY_SV/1/raw/WD_BinaryCandidates_Variability.csv
Fixing string column names
Read 133651 rows
First 3 rows:
     source_id            ra           dec       ...     pmdec      Epoch  epoch
------------------- ------------- -------------- ... -------------- ------ -----
4026850114257127040 181.785874231  32.3654862154 ...  1.71074109659 2015.5    --
5573594388757919616 95.0484717384 -40.0562593775 ...  1.87401199019 2015.5    --
5672222574635034880 148.886907858 -17.6974649742 ... -18.8238754235 2015.5    --

Column names:
['source_id', 'ra', 'dec', 'pmra', 'pmdec', 'Epoch', 'epoch']

Selecting 95174 from 133651 (71.21%) on dec
132995 of 133651 targets have unique source_id
Removing 656 duplicates

Writing 94659 rows of output to /global/projecta/projectdirs/desi/external/mws/sv/